## Homework №2

### Neural machine translation and Attention layer.

Your ultimate tast is to enchance the Day4 architecture for Neural Machine Translation task from German to English.
It consists of two parts.
__After you finish some experiments__, please, write down some notes highlighting your main ideas you've got/problems you've faced/solutions you've found during the experiments. 

##### Part 1. Attention implementation.
Your first task is to implement the attention layer, modify the Decoder and test your new seq2seq model for machine translation. Code in this notebook refers a lot to the Day04 practical class. You have to modify the network from the class by adding `Attention` module to it.

_Optional: if you wish, you could also try the bidirectional LSTM in the Encoder._

##### Part 2. CNN encoder implementation.
The second task is to redesign the `Encoder` to use CNN instead of LSTM. `Attention` is not required in this part.



You may leave all the data loading, data preprocessing parts the same as it was in Day4 practice.

###  Part 1. Attention

Attention layer can take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

You can use any type of the attention scores you prefer. We have met at least three of them:

<img src="img/scoring_functions.png" />


_Some additional notes coming next:_

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src sent len]** tensor for each example in the batch. We want this to be **[src sent len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of this as calculating a weighted sum of the "match" over all `dec_hid_dem` elements for each encoder hidden state, where the weights are learned (as we learn the parameters of $v$).

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/yellow blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](assets/seq2seq9.png)

In [ ]:
# Here is some basic code to start with. You may use it or just rewrite everything in a manner you like.
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = # <YOUR CODE HERE>
        
    def forward(self, hidden, encoder_outputs):
        # <YOUR CODE HERE>
        
        return 

### Decoder

To make it really work you should also change the`Decoder` class from the classwork in order to make it to use `Attention`. 
You may just copy-paste `Decoder` class and add several lines of code to it.

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The input word (that has been embedded), $y_t$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $y_t$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(y_t, w_t, s_{t-1})$$

We then pass $y_t$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(y_t, w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](assets/seq2seq10.png)

The green/yellow blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [ ]:
class DecoderWithAttention(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = # <YOUR CODE HERE>
        
        self.out = # <YOUR CODE HERE>
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        # <YOUR CODE HERE>

Here should go Seq2Seq training pipeline (reuse code from Day 4).

Compare the results. Write down some thoughts about translation quality. Was attention worth adding to the model?

In [ ]:
# your thoughts

### Part 2. CNN encoder.
Now try to modify the `Encoder class` and tune the network again. Try to achieve the score at least comparable to the original translation model without attention layer.

In [ ]:
def CNNEncoder(nn.Module):
    def __init__(self, ...):
        # <YOUR CODE HERE>
    
    def forward(self, ...):
        # <YOUR CODE HERE>

In [1]:
# Here comes the whole pipeline again.

In [ ]:
# <YOUR CODE HERE>

In [2]:
# Your thoughts about the experiments here